In [1]:
import pandas as pd
import numpy as np

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql import functions as f


In [3]:
import findspark
findspark.init()

In [4]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [5]:
import os

### Criando uma sessão do PySpark

In [6]:

# spark = SparkSession.builder \
#             .master('local[*]') \
#             .appName('Iniciando com Spark') \
#             .getOrCreate()

In [7]:
# spark

In [8]:
spark = SparkSession.builder \
            .master('local[*]') \
            .appName('Iniciando com Spark') \
            .config('spark.ui.port', '4050') \
            .getOrCreate()

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 59010)
Traceback (most recent call last):
  File "C:\Users\ferna\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\ferna\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\ferna\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\ferna\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 755, in __init__
    self.handle()
  File "D:\Spark\.venv\Lib\site-packages\pyspark\accumulators.py", line 295, in handle
    poll(accum_updates)
  File "D:\Spark\.venv\Lib\site-packages\pyspark\accumulators.py", line 267, in poll
    if self.rfile in 

Site ngrok


In [9]:
# from pyngrok import ngrok

In [10]:
# !ngrok authtoken "MEU COD"

In [11]:
# ngrok.connect('4050')

In [12]:
data = [('Zeca', '35'), ('Eva', '29')]
colNames = ['Nome', 'Idade']

In [13]:
df = spark.createDataFrame(data, colNames)

In [14]:
df.show()

+----+-----+
|Nome|Idade|
+----+-----+
|Zeca|   35|
| Eva|   29|
+----+-----+



In [15]:
df.toPandas()

,Nome,Idade
0,Zeca,35
1,Eva,29


ZipFile

In [16]:
# import zipfile
# zipfile.ZipFile('datasets/empresas.zip', 'r').extractall('datasets/')

In [17]:
def get_file_paths(dir_name):
    dir_path = os.path.abspath(dir_name)
    res = []
    for path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, path)):
            res.append(os.path.join(dir_path, path))
    return res

# # Obtem os caminhos dos arquivos para cada diretório
estabelecimentos_files = get_file_paths("datasets/estabelecimentos/")
socios_files = get_file_paths("datasets/socios/")
empresas_files = get_file_paths("datasets/empresas/")

# # Lê os arquivos em DataFrames Spark
estabelecimentos_df = spark.read.csv(estabelecimentos_files, sep=';', inferSchema=True)
socios_df = spark.read.csv(socios_files, sep=';', inferSchema=True)
empresas_df = spark.read.csv(empresas_files, sep=';', inferSchema=True)


In [18]:
empresas_df.count(), estabelecimentos_df.count(), socios_df.count()
# 4.585.679, # 4.836.219, # 2.046.430

(4585679, 1934441, 2046430)

Manipulando os dados 

In [19]:
socios_df.limit(5).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10
0,11748,2,MARIO KATUMI HOSI,***504158**,49,19940530,NaN,***000000**,None,0,7
1,11748,2,ROBERTO YUKIO HOSI,***241578**,22,19940530,NaN,***000000**,None,0,7
2,13289,2,ANDREIA CRISTINA DELSIN,***787278**,65,20180615,NaN,***000000**,None,0,3
3,17389,2,MARCIA DO CANTO ARRUDA DAIER,***920408**,49,19940613,NaN,***000000**,None,0,7
4,19204,2,ALMIR CARLOS CAPELLINI,***299028**,49,19980908,NaN,***000000**,None,0,7


In [20]:
empresasColNames = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']
estabsColNames = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']
sociosColNames = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

In [21]:
for index,colName in enumerate(empresasColNames):
        empresas_df = empresas_df.withColumnRenamed(f"_c{index}", colName)

for index,colName in enumerate(estabsColNames):
        estabelecimentos_df = estabelecimentos_df.withColumnRenamed(f"_c{index}", colName)

for index,colName in enumerate(sociosColNames):
        socios_df = socios_df.withColumnRenamed(f"_c{index}", colName)


In [22]:
# empresas_df.columns
# estabelecimentos_df.columns
# socios_df.columns


Analisando os dados

In [23]:
empresas_df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: string (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [24]:
socios_df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: integer (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [25]:
estabelecimentos_df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: integer (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: integer (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: str

Converter String -> Double
<br>
´StringType -> DoubleType´

In [26]:
empresas_df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: string (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [27]:
empresas_df.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,4519,DANIELA DA SILVA CRUZ,2135,50,"0,00",5,None
1,8638,JOAO DOS SANTOS FAGUNDES,2135,50,"0,00",5,None
2,11748,PANIFICADORA E CONFEITARIA CONFIANCA RIO PRETO...,2062,49,"0,00",1,None
3,12027,L G SORVETERIA LTDA,2062,49,"0,00",5,None
4,13289,ANDREIA CRISTINA DELSIN EIRELI,2305,65,"100000,00",1,None


In [28]:
empresas_df = empresas_df.withColumn('capital_social_da_empresa', f.regexp_replace('capital_social_da_empresa', ',', '.'))
empresas_df.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,4519,DANIELA DA SILVA CRUZ,2135,50,0.00,5,None
1,8638,JOAO DOS SANTOS FAGUNDES,2135,50,0.00,5,None
2,11748,PANIFICADORA E CONFEITARIA CONFIANCA RIO PRETO...,2062,49,0.00,1,None
3,12027,L G SORVETERIA LTDA,2062,49,0.00,5,None
4,13289,ANDREIA CRISTINA DELSIN EIRELI,2305,65,100000.00,1,None


In [29]:
empresas_df = empresas_df.withColumn('capital_social_da_empresa', empresas_df['capital_social_da_empresa'].cast(DoubleType()))
empresas_df.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,4519,DANIELA DA SILVA CRUZ,2135,50,0.0,5,None
1,8638,JOAO DOS SANTOS FAGUNDES,2135,50,0.0,5,None
2,11748,PANIFICADORA E CONFEITARIA CONFIANCA RIO PRETO...,2062,49,0.0,1,None
3,12027,L G SORVETERIA LTDA,2062,49,0.0,5,None
4,13289,ANDREIA CRISTINA DELSIN EIRELI,2305,65,100000.0,1,None


In [30]:
empresas_df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [31]:
df = spark.createDataFrame([(20200924,), (20201022,), (20210215,)], ['data'])
df.toPandas()


,data
0,20200924
1,20201022
2,20210215


In [32]:
df.printSchema()

root
 |-- data: long (nullable = true)



In [33]:
df = df.withColumn("data", f.to_date(df['data'].cast(StringType()), 'yyyyMMdd'))
df.printSchema()


root
 |-- data: date (nullable = true)



In [34]:
df.toPandas()

,data
0,2020-09-24
1,2020-10-22
2,2021-02-15


In [35]:
estabelecimentos_df = estabelecimentos_df\
            .withColumn(
            "data_situacao_cadastral",
            f.to_date(estabelecimentos_df.data_situacao_cadastral.cast(StringType()), 'yyyyMMdd')
            )


In [36]:
estabelecimentos_df = estabelecimentos_df\
            .withColumn(
            "data_de_inicio_atividade",
            f.to_date(estabelecimentos_df.data_de_inicio_atividade.cast(StringType()), 'yyyyMMdd')
            )\
            .withColumn(
            "data_da_situacao_especial",
            f.to_date(estabelecimentos_df.data_da_situacao_especial.cast(StringType()), 'yyyyMMdd')
            )


In [37]:
estabelecimentos_df[['data_da_situacao_especial', 'data_de_inicio_atividade', 'data_situacao_cadastral']].limit(10).toPandas()

,data_da_situacao_especial,data_de_inicio_atividade,data_situacao_cadastral
0,None,1994-05-16,1995-03-31
1,None,1994-05-24,2015-02-09
2,None,1994-05-31,2018-12-19
3,None,1994-06-08,2008-12-31
4,None,1994-06-01,2004-01-23
5,None,1973-08-21,2008-12-31
6,None,1994-06-13,2018-10-04
7,None,1994-06-21,1996-05-06
8,None,1994-06-16,2012-10-04
9,None,1994-06-28,2016-10-10


In [38]:
socios_df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: integer (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [39]:
socios_df.limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,11748,2,MARIO KATUMI HOSI,***504158**,49,19940530,NaN,***000000**,None,0,7
1,11748,2,ROBERTO YUKIO HOSI,***241578**,22,19940530,NaN,***000000**,None,0,7
2,13289,2,ANDREIA CRISTINA DELSIN,***787278**,65,20180615,NaN,***000000**,None,0,3
3,17389,2,MARCIA DO CANTO ARRUDA DAIER,***920408**,49,19940613,NaN,***000000**,None,0,7
4,19204,2,ALMIR CARLOS CAPELLINI,***299028**,49,19980908,NaN,***000000**,None,0,7


In [40]:
socios_df = socios_df.withColumn('data_de_entrada_sociedade', f.to_date(socios_df.data_de_entrada_sociedade.cast(StringType()), 'yyyyMMdd'))

# Seleções e consultas

### Selecionando informações

In [41]:
empresas_df \
    .select('*') \
    .show(5)

+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial|natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|       4519|         DANIELA DA SILVA ...|             2135|                         50|                      0.0|               5|                       NULL|
|       8638|         JOAO DOS SANTOS F...|             2135|                         50|                      0.0|               5|                       NULL|
|      11748|         PANIFICADORA E CO...|             2062|                         49|                      0.0|               1|                       NULL|
|      12027|          L G SORVETE

In [42]:
empresas_df \
    .select('*') \
    .show(5, False)

+-----------+---------------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial                      |natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+---------------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|4519       |DANIELA DA SILVA CRUZ                              |2135             |50                         |0.0                      |5               |NULL                       |
|8638       |JOAO DOS SANTOS FAGUNDES                           |2135             |50                         |0.0                      |5               |NULL                       |
|11748      |PANIFICADORA E CONFEITARIA CONFIANCA RIO PRETO LTDA|2062             |49

In [43]:
empresas_df \
    .select('natureza_juridica', 'porte_da_empresa', 'capital_social_da_empresa') \
    .show(5)

+-----------------+----------------+-------------------------+
|natureza_juridica|porte_da_empresa|capital_social_da_empresa|
+-----------------+----------------+-------------------------+
|             2135|               5|                      0.0|
|             2135|               5|                      0.0|
|             2062|               1|                      0.0|
|             2062|               5|                      0.0|
|             2305|               1|                 100000.0|
+-----------------+----------------+-------------------------+
only showing top 5 rows



In [44]:
socios_df \
    .select('nome_do_socio_ou_razao_social', 'faixa_etaria',  f.year('data_de_entrada_sociedade').alias('ano_de_entrada'), f.month('data_de_entrada_sociedade').alias('mes_de_entrada'), f.day('data_de_entrada_sociedade').alias('dia_de_entrada')) \
    .show(5, False)

+-----------------------------+------------+--------------+--------------+--------------+
|nome_do_socio_ou_razao_social|faixa_etaria|ano_de_entrada|mes_de_entrada|dia_de_entrada|
+-----------------------------+------------+--------------+--------------+--------------+
|MARIO KATUMI HOSI            |7           |1994          |5             |30            |
|ROBERTO YUKIO HOSI           |7           |1994          |5             |30            |
|ANDREIA CRISTINA DELSIN      |3           |2018          |6             |15            |
|MARCIA DO CANTO ARRUDA DAIER |7           |1994          |6             |13            |
|ALMIR CARLOS CAPELLINI       |7           |1998          |9             |8             |
+-----------------------------+------------+--------------+--------------+--------------+
only showing top 5 rows



In [45]:
estabelecimentos_df \
    .select('nome_fantasia', 'municipio', f.year('data_de_inicio_atividade').alias('ano_inicio_atividade'), f.month('data_de_inicio_atividade').alias('mes_inicio_atividade')) \
    .show(5)


+--------------------+---------+--------------------+--------------------+
|       nome_fantasia|municipio|ano_inicio_atividade|mes_inicio_atividade|
+--------------------+---------+--------------------+--------------------+
|            GIRAFFAS|     6219|                1994|                   5|
|AGROPECUARIA FAGU...|     7255|                1994|                   5|
|                NULL|     7097|                1994|                   5|
|                NULL|     7107|                1994|                   6|
|JS MATERIAIS DE C...|     6915|                1994|                   6|
+--------------------+---------+--------------------+--------------------+
only showing top 5 rows



In [46]:
data = [
    ('GISELLE PAULA GUIMARAES CASTRO', 15),
    ('ELAINE GARCIA DE OLIVEIRA', 22),
    ('JOAO CARLOS ABNER DE LOURDES', 43),
    ('MARTA ZELI FERREIRA', 24),
    ('LAUDENETE WIGGERS ROEDER', 51)
]
colNames = ['nome', 'idade']
df = spark.createDataFrame(data, colNames)
df.show(truncate=False)

+------------------------------+-----+
|nome                          |idade|
+------------------------------+-----+
|GISELLE PAULA GUIMARAES CASTRO|15   |
|ELAINE GARCIA DE OLIVEIRA     |22   |
|JOAO CARLOS ABNER DE LOURDES  |43   |
|MARTA ZELI FERREIRA           |24   |
|LAUDENETE WIGGERS ROEDER      |51   |
+------------------------------+-----+



In [47]:
df \
    .select(
        f.concat_ws(
            ', ', 
            f.substring_index('nome', ' ', -1), 
            f.substring_index('nome', ' ', 1)
        ).alias('ident'), 
        'idade') \
    .show()

+-----------------+-----+
|            ident|idade|
+-----------------+-----+
|  CASTRO, GISELLE|   15|
| OLIVEIRA, ELAINE|   22|
|    LOURDES, JOAO|   43|
|  FERREIRA, MARTA|   24|
|ROEDER, LAUDENETE|   51|
+-----------------+-----+



In [48]:
df \
    .select(
        f.concat_ws(
            ', ', 
            f.substring_index('nome', ' ', -1), 
            f.substring_index('nome', ' ', 1)
        ).alias('ident'), 
        'idade') \
    .show(truncate=False)

+-----------------+-----+
|ident            |idade|
+-----------------+-----+
|CASTRO, GISELLE  |15   |
|OLIVEIRA, ELAINE |22   |
|LOURDES, JOAO    |43   |
|FERREIRA, MARTA  |24   |
|ROEDER, LAUDENETE|51   |
+-----------------+-----+



# Identificando valores Nulos

In [49]:
df = spark.createDataFrame([(1, ), (2, ), (3, ), (None, )], ['data'])
df.toPandas()

,data
0,1.0
1,2.0
2,3.0
3,NaN


In [50]:
df.show()

+----+
|data|
+----+
|   1|
|   2|
|   3|
|NULL|
+----+



In [51]:
df = spark.createDataFrame([(1.,), (2.,), (3.,), (float('nan'), )], ['data'])
df.toPandas()

,data
0,1.0
1,2.0
2,3.0
3,NaN


In [52]:
df.show()

+----+
|data|
+----+
| 1.0|
| 2.0|
| 3.0|
| NaN|
+----+



In [53]:
df = spark.createDataFrame([(1.,), (2.,), (3.,), (float('nan'), )], ['data'])
df.show()

+----+
|data|
+----+
| 1.0|
| 2.0|
| 3.0|
| NaN|
+----+



In [54]:
df.show()

+----+
|data|
+----+
| 1.0|
| 2.0|
| 3.0|
| NaN|
+----+



In [55]:
df = spark.createDataFrame([('1',), ('2',), ('3',), (None,)], ['data'])
df.toPandas()

,data
0,1
1,2
2,3
3,None


In [56]:
df.show()

+----+
|data|
+----+
|   1|
|   2|
|   3|
|NULL|
+----+



In [57]:
socios_df.limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,11748,2,MARIO KATUMI HOSI,***504158**,49,1994-05-30,NaN,***000000**,None,0,7
1,11748,2,ROBERTO YUKIO HOSI,***241578**,22,1994-05-30,NaN,***000000**,None,0,7
2,13289,2,ANDREIA CRISTINA DELSIN,***787278**,65,2018-06-15,NaN,***000000**,None,0,3
3,17389,2,MARCIA DO CANTO ARRUDA DAIER,***920408**,49,1994-06-13,NaN,***000000**,None,0,7
4,19204,2,ALMIR CARLOS CAPELLINI,***299028**,49,1998-09-08,NaN,***000000**,None,0,7


In [58]:
socios_df[['pais', 'nome_do_representante']].limit(5).show()

+----+---------------------+
|pais|nome_do_representante|
+----+---------------------+
|NULL|                 NULL|
|NULL|                 NULL|
|NULL|                 NULL|
|NULL|                 NULL|
|NULL|                 NULL|
+----+---------------------+



In [59]:
socios_df.select( [f.count(f.when(f.isnull(c), 1)).alias(c) for c in socios_df.columns] ).show()

+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+-------+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social|cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|   pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+-------+-------------------+---------------------+-----------------------------------+------------+
|          0|                     0|                          208|                1234|                    0|                        1|2038255|                  0|              1995432|                                  0|           0|
+-----------+----------------------+------------------------

In [60]:
socios_df.fillna(0).limit(5).show()

+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social|cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|      11748|                     2|            MARIO KATUMI HOSI|         ***504158**|                   49|               1994-05-30|   0|        ***000000**|                 NULL|                                  0|           7|
|      11748|                     2|           ROBERTO YUKIO HOSI|      

In [61]:
socios_df.fillna('-').limit(5).show()

+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social|cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|      11748|                     2|            MARIO KATUMI HOSI|         ***504158**|                   49|               1994-05-30|NULL|        ***000000**|                    -|                                  0|           7|
|      11748|                     2|           ROBERTO YUKIO HOSI|      

# Ordenando os dados

In [62]:
socios_df \
    .select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .orderBy('ano_de_entrada', ascending=False) \
    .show(5, False)


+------------------------------+------------+--------------+
|nome_do_socio_ou_razao_social |faixa_etaria|ano_de_entrada|
+------------------------------+------------+--------------+
|MARA LUCIETI MICHEL           |5           |2021          |
|ANA LUCIA ALVES DE CARVALHO   |5           |2021          |
|DOUGLAS CAPPELLETTI           |3           |2021          |
|JORGE RODRIGUES SANTOS        |6           |2021          |
|FLAVIO SOARES FERREIRA MARTINS|6           |2021          |
+------------------------------+------------+--------------+
only showing top 5 rows



In [63]:
socios_df \
    .select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .orderBy(['ano_de_entrada', 'faixa_etaria'], ascending=[False, True]) \
    .show(15, False)


+------------------------------------------------------------+------------+--------------+
|nome_do_socio_ou_razao_social                               |faixa_etaria|ano_de_entrada|
+------------------------------------------------------------+------------+--------------+
|CONDOMINIO SOLAR APOLO SPE LTDA                             |0           |2021          |
|CONTERSOLO CONSTRUTORA DE OBRAS EIRELI                      |0           |2021          |
|COMPANHIA BRASILEIRA DE SERVICOS FUNERARIOS LTDA            |0           |2021          |
|RUSSELL BEDFORD BRASIL AUDITORES INDEPENDENTES S/S          |0           |2021          |
|AGROSYSTEM INDUSTRIA, COMERCIO, IMPORTACAO E EXPORTACAO LTDA|0           |2021          |
|TRISUL S.A.                                                 |0           |2021          |
|TERRABEL EMPREENDIMENTOS LTDA                               |0           |2021          |
|WEILLER CONSTRUCAO CIVIL LTDA                               |0           |2021          |

# Filtrando os dados

In [64]:
empresas_df \
    .where('capital_social_da_empresa==50') \
    .show(5, False)


+-----------+--------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial               |natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+--------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|14715041   |LIDIANE MARIA DO NASCIMENTO 14542418707     |2135             |50                         |50.0                     |1               |NULL                       |
|20601885   |CRISTIANO AKIHITO BORDIN 04370949955        |2135             |50                         |50.0                     |1               |NULL                       |
|23661983   |VITOR ALOISIO DO NASCIMENTO GUIA 12663882739|2135             |50                         |50.0            

In [65]:
socios_df \
    .select('nome_do_socio_ou_razao_social') \
    .filter(socios_df.nome_do_socio_ou_razao_social.startswith('FERNANDO'))\
    .filter(socios_df.nome_do_socio_ou_razao_social.endswith('BATISTA'))\
    .limit(10)\
    .toPandas()



,nome_do_socio_ou_razao_social
0,FERNANDO SOARES BATISTA
1,FERNANDO CESAR BATISTA
2,FERNANDO AUGUSTO LOPES BATISTA
3,FERNANDO BATISTA
4,FERNANDO RIBEIRO BATISTA
5,FERNANDO BRAGA BATISTA
6,FERNANDO PINTO BATISTA
7,FERNANDO SANTANA BATISTA
8,FERNANDO SANTOS BATISTA
9,FERNANDO JOSE NEVES BATISTA


In [66]:
data = [
    ('CARMINA RABELO', 4, 2010), 
    ('HERONDINA PEREIRA', 6, 2009), 
    ('IRANI DOS SANTOS', 12, 2010), 
    ('JOAO BOSCO DA FONSECA', 3, 2009), 
    ('CARLITO SOUZA', 1, 2010), 
    ('WALTER DIAS', 9, 2009), 
    ('BRENO VENTUROSO', 1, 2009), 
    ('ADELINA TEIXEIRA', 5, 2009), 
    ('ELIO SILVA', 7, 2010), 
    ('DENIS FONSECA', 6, 2010)
]
colNames = ['nome', 'mes', 'ano']
df = spark.createDataFrame(data, colNames)
df.show(truncate=False)

+---------------------+---+----+
|nome                 |mes|ano |
+---------------------+---+----+
|CARMINA RABELO       |4  |2010|
|HERONDINA PEREIRA    |6  |2009|
|IRANI DOS SANTOS     |12 |2010|
|JOAO BOSCO DA FONSECA|3  |2009|
|CARLITO SOUZA        |1  |2010|
|WALTER DIAS          |9  |2009|
|BRENO VENTUROSO      |1  |2009|
|ADELINA TEIXEIRA     |5  |2009|
|ELIO SILVA           |7  |2010|
|DENIS FONSECA        |6  |2010|
+---------------------+---+----+



In [67]:
df\
    .filter("mes<=6")\
    .filter("ano=2009")\
    .show(truncate=False)

+---------------------+---+----+
|nome                 |mes|ano |
+---------------------+---+----+
|HERONDINA PEREIRA    |6  |2009|
|JOAO BOSCO DA FONSECA|3  |2009|
|BRENO VENTUROSO      |1  |2009|
|ADELINA TEIXEIRA     |5  |2009|
+---------------------+---+----+



In [68]:
df\
    .filter("mes<=6 and ano==2009")\
    .show(truncate=False)

+---------------------+---+----+
|nome                 |mes|ano |
+---------------------+---+----+
|HERONDINA PEREIRA    |6  |2009|
|JOAO BOSCO DA FONSECA|3  |2009|
|BRENO VENTUROSO      |1  |2009|
|ADELINA TEIXEIRA     |5  |2009|
+---------------------+---+----+



In [69]:
df\
    .filter(df.mes <= 6)\
    .filter(df.ano == 2009)\
    .show(truncate=False)

+---------------------+---+----+
|nome                 |mes|ano |
+---------------------+---+----+
|HERONDINA PEREIRA    |6  |2009|
|JOAO BOSCO DA FONSECA|3  |2009|
|BRENO VENTUROSO      |1  |2009|
|ADELINA TEIXEIRA     |5  |2009|
+---------------------+---+----+



In [70]:
df\
    .filter((df.mes <= 6) & (df.ano == 2009))\
    .show(truncate=False)

+---------------------+---+----+
|nome                 |mes|ano |
+---------------------+---+----+
|HERONDINA PEREIRA    |6  |2009|
|JOAO BOSCO DA FONSECA|3  |2009|
|BRENO VENTUROSO      |1  |2009|
|ADELINA TEIXEIRA     |5  |2009|
+---------------------+---+----+



# Comando LIKE

In [71]:
df = spark.createDataFrame([('RESTAURANTE DO RUI',), ('Juca restaurantes ltda',), ('Joca Restaurante',)], ['data'])
df.toPandas()

,data
0,RESTAURANTE DO RUI
1,Juca restaurantes ltda
2,Joca Restaurante


In [72]:
df\
    .where(f.upper(df.data).like('RESTAURANTE%'))\
    .show(truncate=False)

+------------------+
|data              |
+------------------+
|RESTAURANTE DO RUI|
+------------------+



In [73]:
empresas_df \
    .select('razao_social_nome_empresarial', 'natureza_juridica', 'porte_da_empresa', 'capital_social_da_empresa') \
    .filter(f.lower(empresas_df.razao_social_nome_empresarial).like('restaurante%')) \
    .show(15, False)

+-----------------------------------------+-----------------+----------------+-------------------------+
|razao_social_nome_empresarial            |natureza_juridica|porte_da_empresa|capital_social_da_empresa|
+-----------------------------------------+-----------------+----------------+-------------------------+
|RESTAURANTE E PETISQUERIA MANE CANTO LTDA|2062             |5               |0.0                      |
|RESTAURANTE E LANCHONETE NOVAES LTDA     |2062             |1               |0.0                      |
|RESTAURANTE GALEAO 2420 LTDA             |2062             |1               |0.0                      |
|RESTAURANTE DIVINO SABOR LTDA            |2062             |1               |0.0                      |
|RESTAURANTE E PIZZARIA LUIS GOES LTDA    |2062             |1               |0.0                      |
|RESTAURANTE E PIZZARIA NOTA DEZ LTDA     |2062             |1               |5000.0                   |
|RESTAURANTE SO CHURRASCO LTDA            |2062        

In [74]:
empresas_df \
    .select('razao_social_nome_empresarial', 'natureza_juridica', 'porte_da_empresa', 'capital_social_da_empresa') \
    .filter(f.upper(empresas_df['razao_social_nome_empresarial']).like('%RESTAURANTE%')) \
    .show(15, False)

+-----------------------------------------------------+-----------------+----------------+-------------------------+
|razao_social_nome_empresarial                        |natureza_juridica|porte_da_empresa|capital_social_da_empresa|
+-----------------------------------------------------+-----------------+----------------+-------------------------+
|PRAIA MAR RESTAURANTE LTDA                           |2062             |5               |20000.0                  |
|MORENO'S RESTAURANTE E LANCHONETE LTDA               |2062             |1               |0.0                      |
|RESTAURANTE E PETISQUERIA MANE CANTO LTDA            |2062             |5               |0.0                      |
|J.C.DE OLIVEIRA RESTAURANTE                          |2135             |5               |0.0                      |
|RESTAURANTE E LANCHONETE NOVAES LTDA                 |2062             |1               |0.0                      |
|RESTAURANTE GALEAO 2420 LTDA                         |2062     

# Agregações e Junções

## Sumarizando os dados

In [75]:
socios_df\
    .select(f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .where("ano_de_entrada >= 2010")\
    .groupBy("ano_de_entrada")\
    .count()\
    .orderBy('ano_de_entrada', ascending=True)\
    .show()

+--------------+------+
|ano_de_entrada| count|
+--------------+------+
|          2010| 79337|
|          2011| 83906|
|          2012| 80101|
|          2013| 83919|
|          2014| 80590|
|          2015| 80906|
|          2016| 81587|
|          2017| 90221|
|          2018| 99935|
|          2019|118248|
|          2020|125927|
|          2021| 56316|
+--------------+------+



In [76]:
empresas_df\
    .select('cnpj_basico', 'porte_da_empresa', 'capital_social_da_empresa')\
    .groupBy('porte_da_empresa')\
    .agg(
        f.avg("capital_social_da_empresa").alias("capital_social_medio"),
        f.count("cnpj_basico").alias("frequencia")
    )\
    .orderBy('porte_da_empresa', ascending=True)\
    .show()

+----------------+--------------------+----------+
|porte_da_empresa|capital_social_medio|frequencia|
+----------------+--------------------+----------+
|            NULL|    8.35421888053467|      5985|
|               1|  339994.53313507047|   3129043|
|               3|  2601001.7677092687|    115151|
|               5|   708660.4208249794|   1335500|
+----------------+--------------------+----------+



In [77]:
empresas_df\
    .select('capital_social_da_empresa')\
    .summary()\
    .show()
# .summary("count","mean","stddev","min","25%","50%","75%","max")

+-------+-------------------------+
|summary|capital_social_da_empresa|
+-------+-------------------------+
|  count|                  4585679|
|   mean|       503694.54785426764|
| stddev|      2.111869149053764E8|
|    min|                      0.0|
|    25%|                      0.0|
|    50%|                   1000.0|
|    75%|                   7000.0|
|    max|         3.22014670262E11|
+-------+-------------------------+



## A função `when`

In [78]:
data = [
    ('CARLOS', 'MATEMÁTICA', 7), 
    ('IVO', 'MATEMÁTICA', 9), 
    ('MÁRCIA', 'MATEMÁTICA', 8), 
    ('LEILA', 'MATEMÁTICA', 9), 
    ('BRENO', 'MATEMÁTICA', 7), 
    ('LETÍCIA', 'MATEMÁTICA', 8), 
    ('CARLOS', 'FÍSICA', 2), 
    ('IVO', 'FÍSICA', 8), 
    ('MÁRCIA', 'FÍSICA', 10), 
    ('LEILA', 'FÍSICA', 9), 
    ('BRENO', 'FÍSICA', 1), 
    ('LETÍCIA', 'FÍSICA', 6), 
    ('CARLOS', 'QUÍMICA', 10), 
    ('IVO', 'QUÍMICA', 8), 
    ('MÁRCIA', 'QUÍMICA', 1), 
    ('LEILA', 'QUÍMICA', 10), 
    ('BRENO', 'QUÍMICA', 7), 
    ('LETÍCIA', 'QUÍMICA', 9)
]
colNames = ['nome', 'materia', 'nota']
df = spark.createDataFrame(data, colNames)
df.show()

+-------+----------+----+
|   nome|   materia|nota|
+-------+----------+----+
| CARLOS|MATEMÁTICA|   7|
|    IVO|MATEMÁTICA|   9|
| MÁRCIA|MATEMÁTICA|   8|
|  LEILA|MATEMÁTICA|   9|
|  BRENO|MATEMÁTICA|   7|
|LETÍCIA|MATEMÁTICA|   8|
| CARLOS|    FÍSICA|   2|
|    IVO|    FÍSICA|   8|
| MÁRCIA|    FÍSICA|  10|
|  LEILA|    FÍSICA|   9|
|  BRENO|    FÍSICA|   1|
|LETÍCIA|    FÍSICA|   6|
| CARLOS|   QUÍMICA|  10|
|    IVO|   QUÍMICA|   8|
| MÁRCIA|   QUÍMICA|   1|
|  LEILA|   QUÍMICA|  10|
|  BRENO|   QUÍMICA|   7|
|LETÍCIA|   QUÍMICA|   9|
+-------+----------+----+



In [79]:
df = df.withColumn('status', f.when(df.nota > 8, "OS MELHORES").when(df.nota >= 5, "APROVADO").otherwise("REPROVADO"))
df.show()

+-------+----------+----+-----------+
|   nome|   materia|nota|     status|
+-------+----------+----+-----------+
| CARLOS|MATEMÁTICA|   7|   APROVADO|
|    IVO|MATEMÁTICA|   9|OS MELHORES|
| MÁRCIA|MATEMÁTICA|   8|   APROVADO|
|  LEILA|MATEMÁTICA|   9|OS MELHORES|
|  BRENO|MATEMÁTICA|   7|   APROVADO|
|LETÍCIA|MATEMÁTICA|   8|   APROVADO|
| CARLOS|    FÍSICA|   2|  REPROVADO|
|    IVO|    FÍSICA|   8|   APROVADO|
| MÁRCIA|    FÍSICA|  10|OS MELHORES|
|  LEILA|    FÍSICA|   9|OS MELHORES|
|  BRENO|    FÍSICA|   1|  REPROVADO|
|LETÍCIA|    FÍSICA|   6|   APROVADO|
| CARLOS|   QUÍMICA|  10|OS MELHORES|
|    IVO|   QUÍMICA|   8|   APROVADO|
| MÁRCIA|   QUÍMICA|   1|  REPROVADO|
|  LEILA|   QUÍMICA|  10|OS MELHORES|
|  BRENO|   QUÍMICA|   7|   APROVADO|
|LETÍCIA|   QUÍMICA|   9|OS MELHORES|
+-------+----------+----+-----------+



## Juntando DataFrames - Joins

In [80]:
produtos = spark.createDataFrame(
    [
        ('1', 'Bebidas', 'Água mineral'), 
        ('2', 'Limpeza', 'Sabão em pó'), 
        ('3', 'Frios', 'Queijo'), 
        ('4', 'Bebidas', 'Refrigerante'),
        ('5', 'Pet', 'Ração para cães')
    ],
    ['id', 'cat', 'prod']
)

impostos = spark.createDataFrame(
    [
        ('Bebidas', 0.15), 
        ('Limpeza', 0.05),
        ('Frios', 0.065),
        ('Carnes', 0.08)
    ],
    ['cat', 'tax']
)

In [81]:
produtos.toPandas()

,id,cat,prod
0,1,Bebidas,Água mineral
1,2,Limpeza,Sabão em pó
2,3,Frios,Queijo
3,4,Bebidas,Refrigerante
4,5,Pet,Ração para cães


In [82]:
impostos.toPandas()

,cat,tax
0,Bebidas,0.150
1,Limpeza,0.050
2,Frios,0.065
3,Carnes,0.080


In [83]:
produtos.join(impostos, on='cat', how='inner').sort('id').show()

+-------+---+------------+-----+
|    cat| id|        prod|  tax|
+-------+---+------------+-----+
|Bebidas|  1|Água mineral| 0.15|
|Limpeza|  2| Sabão em pó| 0.05|
|  Frios|  3|      Queijo|0.065|
|Bebidas|  4|Refrigerante| 0.15|
+-------+---+------------+-----+



In [84]:
produtos.join(impostos, on='cat', how='left').sort('id').show()

+-------+---+---------------+-----+
|    cat| id|           prod|  tax|
+-------+---+---------------+-----+
|Bebidas|  1|   Água mineral| 0.15|
|Limpeza|  2|    Sabão em pó| 0.05|
|  Frios|  3|         Queijo|0.065|
|Bebidas|  4|   Refrigerante| 0.15|
|    Pet|  5|Ração para cães| NULL|
+-------+---+---------------+-----+



In [85]:
produtos.join(impostos, on='cat', how='right').sort('id').show()

+-------+----+------------+-----+
|    cat|  id|        prod|  tax|
+-------+----+------------+-----+
| Carnes|NULL|        NULL| 0.08|
|Bebidas|   1|Água mineral| 0.15|
|Limpeza|   2| Sabão em pó| 0.05|
|  Frios|   3|      Queijo|0.065|
|Bebidas|   4|Refrigerante| 0.15|
+-------+----+------------+-----+



In [86]:
produtos.join(impostos, on='cat', how='outer').sort('id').show()

+-------+----+---------------+-----+
|    cat|  id|           prod|  tax|
+-------+----+---------------+-----+
| Carnes|NULL|           NULL| 0.08|
|Bebidas|   1|   Água mineral| 0.15|
|Limpeza|   2|    Sabão em pó| 0.05|
|  Frios|   3|         Queijo|0.065|
|Bebidas|   4|   Refrigerante| 0.15|
|    Pet|   5|Ração para cães| NULL|
+-------+----+---------------+-----+



In [87]:
empresas_df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [88]:
socios_df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: date (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [89]:
estabelecimentos_df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: string (n

In [90]:
empresa_join = estabelecimentos_df.join(empresas_df, 'cnpj_basico', how='inner')

In [91]:
empresa_join.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: string (n

In [92]:
freq = empresa_join\
    .select(
        'cnpj_basico',
        f.year(empresa_join.data_de_inicio_atividade).alias('data_de_inicio')
    )\
    .where('data_de_inicio >= 2010')\
    .groupBy('data_de_inicio')\
    .agg(f.count('cnpj_basico').alias('frequencia'))\
    .orderBy('data_de_inicio', ascending=True)


In [93]:
freq.toPandas()

,data_de_inicio,frequencia
0,2010,61987
1,2011,68498
2,2012,92932
3,2013,79497
4,2014,80920
5,2015,85268
6,2016,106863
7,2017,94947
8,2018,110280
9,2019,130324


In [94]:
freq.union(
    freq.select(
        f.lit('Total').alias('data_de_inicio_2'),
        f.sum(freq.frequencia).alias('frequencia_2')
    )
).show()

+--------------+----------+
|data_de_inicio|frequencia|
+--------------+----------+
|          2010|     61987|
|          2011|     68498|
|          2012|     92932|
|          2013|     79497|
|          2014|     80920|
|          2015|     85268|
|          2016|    106863|
|          2017|     94947|
|          2018|    110280|
|          2019|    130324|
|          2020|    160007|
|          2021|     61409|
|         Total|   1132932|
+--------------+----------+



## SparkSQL

In [95]:
empresas_df.createOrReplaceTempView("empresasView")

In [96]:
spark.sql("SELECT * FROM empresasView").show(5, False)

+-----------+---------------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial                      |natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+---------------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|4519       |DANIELA DA SILVA CRUZ                              |2135             |50                         |0.0                      |5               |NULL                       |
|8638       |JOAO DOS SANTOS FAGUNDES                           |2135             |50                         |0.0                      |5               |NULL                       |
|11748      |PANIFICADORA E CONFEITARIA CONFIANCA RIO PRETO LTDA|2062             |49

In [97]:
spark.sql("""
    SELECT * FROM empresasView
    WHERE capital_social_da_empresa = 50
""").show(5, False)

+-----------+--------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial               |natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+--------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|14715041   |LIDIANE MARIA DO NASCIMENTO 14542418707     |2135             |50                         |50.0                     |1               |NULL                       |
|20601885   |CRISTIANO AKIHITO BORDIN 04370949955        |2135             |50                         |50.0                     |1               |NULL                       |
|23661983   |VITOR ALOISIO DO NASCIMENTO GUIA 12663882739|2135             |50                         |50.0            

In [98]:
empresa_join.createOrReplaceTempView('empresaJoinView')

In [99]:
freq = spark\
    .sql("""
        SELECT YEAR(data_de_inicio_atividade) AS data_de_inicio, COUNT(cnpj_basico) AS count
            FROM empresaJoinView 
            WHERE YEAR(data_de_inicio_atividade) >= 2010
            GROUP BY data_de_inicio
            ORDER BY data_de_inicio
    """)

freq\
    .show()

+--------------+------+
|data_de_inicio| count|
+--------------+------+
|          2010| 61987|
|          2011| 68498|
|          2012| 92932|
|          2013| 79497|
|          2014| 80920|
|          2015| 85268|
|          2016|106863|
|          2017| 94947|
|          2018|110280|
|          2019|130324|
|          2020|160007|
|          2021| 61409|
+--------------+------+



In [100]:
freq.createOrReplaceTempView("freqView")

In [101]:
spark\
    .sql("""
        SELECT *
            FROM freqView
        UNION ALL
        SELECT 'Total' AS data_de_inicio, SUM(count) AS count
            FROM freqView
    """)\
    .show()

+--------------+-------+
|data_de_inicio|  count|
+--------------+-------+
|          2010|  61987|
|          2011|  68498|
|          2012|  92932|
|          2013|  79497|
|          2014|  80920|
|          2015|  85268|
|          2016| 106863|
|          2017|  94947|
|          2018| 110280|
|          2019| 130324|
|          2020| 160007|
|          2021|  61409|
|         Total|1132932|
+--------------+-------+



# Formas de armazenamentos

## Arquivos CSV

In [105]:
# empresas_df.write.csv(
#     path='./datasets/csv_extract',
#     mode='overwrite',
#     sep=';',
#     header=True
# )

In [113]:
# empresas2 = spark.write.csv(
#     './datasets/csv_extract',
#     sep=';',
#     inferSchema=True,
#     header=True
# )

In [115]:
# empresas2 = spark.read.csv(
#     './datasets/csv_extract/'
# )

## Arquivos PARQUET

In [111]:
# empresas_parquet = spark.write.parquet(
#     path='./datasets/parquet_extract',
#     mode='overwrite'
# )

In [112]:
# empresas_parquet = spark.read.parquet(
#     './datasets/parquet_extract/'
# )
